In [1]:
import enum
import sys
from graphkitlearn.graphkitlearn.gklearn.utils import graphfiles
import networkx
import matplotlib.pyplot as plt
import numpy as np
from gklearn.utils import *
import os
import random
from gklearn.ged.env import GEDEnv
import numpy as np
from time import process_time
from sklearn.cluster import KMeans
from sklearn.metrics import *
import datetime
pathGrec = "./GREC/GREC/data/"
import pickle

In [2]:
def LoadData(filename, childrentagName):
    import xml.etree.ElementTree as ET
    dirname_dataset = os.path.dirname(filename)
    tree = ET.parse(filename)
    root = tree.getroot()
    data = []
    y = []
    children = list([elem for elem in root.find(childrentagName).iter() if elem is not root.find(childrentagName)])
    for graph in children:
        mol_filename = graph.attrib['file']
        mol_class = graph.attrib['class']
        data.append(graphfiles.loadGXL(dirname_dataset + '/' + mol_filename))
        y.append(mol_class)
    return data, y

In [3]:
class GraphHelper:
    def __init__(self, trainData, mProtoTypes):
        self.trainData = trainData
        self.InitializeGraphToVector(trainData, mProtoTypes)
        self.mProtoTypes = mProtoTypes
        
    def GetDistanceBetweenGraphs(self,graph1, graph2):
        ged_env = GEDEnv() # initailize GED environment.
        ged_env.set_edit_cost('CONSTANT', # GED cost type.
                            edit_cost_constants=[3, 3, 1, 3, 3, 1] # edit costs.
                            )  
        ged_env.add_nx_graph(graph1, '') # add graph1
        ged_env.add_nx_graph(graph2, '') # add graph2
        listID = ged_env.get_all_graph_ids() # get list IDs of graphs
        ged_env.init(init_type='LAZY_WITHOUT_SHUFFLED_COPIES') # initialize GED environment.
        options = {'initialization_method': 'RANDOM', # or 'NODE', etc.
                'threads': 1 # parallel threads.
                }
        ged_env.set_method('BIPARTITE', # GED method.
                        options # options for GED method.
                        )
        ged_env.init_method() # initialize GED method.

        ged_env.run_method(listID[0], listID[1]) # run.
        dis = ged_env.get_upper_bound(listID[0], listID[1])
        return dis

    def MaxEditDistance(self, graphSets, nodes, addedIndices):
        distanceVector = np.empty(shape=(len(graphSets ), len(nodes)))
        for graphIndex,graph in enumerate(graphSets):
            for nodeIndex,node in enumerate(nodes):
                dis = self.GetDistanceBetweenGraphs(graph, node)
                distanceVector[graphIndex][nodeIndex] = dis
        maxValue = -1
        maxIndex = -1
        for graphIndex in range(len(graphSets)):
            if(graphIndex not in addedIndices):
                maxDistanceIndex = np.argmax(distanceVector[graphIndex])
                if(distanceVector[graphIndex][maxDistanceIndex] > maxValue):
                    maxValue = distanceVector[graphIndex][maxDistanceIndex]
                    maxIndex = graphIndex

        return maxIndex

    def SelectSpanningPrototypes(self, graphData, mprototypes):
        choiceIndex = random.randrange(len(graphData))
        graphSelected = [graphData[choiceIndex]]
        graphSelectedIndex = [choiceIndex]

        for selectors in range(mprototypes - 1):
            maxEditDistanceIndex = self.MaxEditDistance(graphData, graphSelected, graphSelectedIndex)
            graphSelectedIndex.append(maxEditDistanceIndex)
            graphSelected.append(graphData[maxEditDistanceIndex])
        return graphSelectedIndex
    
    def GraphToVector(self, graphSet):
        vectorMatrix = np.empty(shape= (len(graphSet), self.mProtoTypes))
        for row, graph in enumerate(graphSet):
            for col,prototypeIndex in enumerate(self.selectedProtoTypes):
                vectorMatrix[row][col] = self.GetDistanceBetweenGraphs(graph, self.trainData[prototypeIndex])
        return vectorMatrix
    
    def InitializeGraphToVector(self, graphSet, mprotoTypes):
        self.selectedProtoTypes = self.SelectSpanningPrototypes(graphSet, mprotoTypes)
        #return self.GraphToVector(graphSet)

In [4]:
XtrainGREC, y_train = LoadData(pathGrec +"train.cxl", "grec")
XvalidateGREC, y_validate = LoadData(pathGrec +"valid.cxl", "grec")
XtestGREC, y_test = LoadData(pathGrec +"test.cxl", "grec")

In [5]:
t1_start = process_time() 
graph = GraphHelper(XtrainGREC, 10)
t1_stop = process_time()
print(t1_stop - t1_start)
now = datetime.datetime.now()
print("Completed Prototype Selection at ", str(now))

290.578125
Completed Prototype Selection at  2021-04-24 15:59:14.563900


In [6]:
t1_start = process_time() 
trainVector = graph.GraphToVector(XtrainGREC)
t1_stop = process_time()
print(t1_stop - t1_start)
now = datetime.datetime.now()
print("Completed Train Conversion at ", str(now))

69.34375
Completed Train Conversion at  2021-04-24 16:00:24.274217


In [7]:
t1_start = process_time() 
testVector = graph.GraphToVector(XtestGREC)
t1_stop = process_time()
print(t1_stop - t1_start)
now = datetime.datetime.now()
print("Completed Test at ", str(now))



139.28125
Completed Test at  2021-04-24 16:02:44.848174


In [8]:
t1_start = process_time() 
validationVector = graph.GraphToVector(XvalidateGREC)
t1_stop = process_time()
print(t1_stop - t1_start)
now = datetime.datetime.now()
print("Completed Validation at ", str(now))


68.359375
Completed Validation at  2021-04-24 16:03:53.524782


## Saving the computed Data

In [9]:
np.save("TrainVectorGREC", trainVector)
np.save("TestVectorGREC", testVector)
np.save("validateVectorGREC", validationVector)
with open('GraphHelperObjGREC', 'wb') as config_dictionary_file:
    pickle.dump(graph, config_dictionary_file)

## Loading from saved files 


In [17]:
trainVector = np.load("TrainVectorGREC.npy")
testVector = np.load("TestVectorGREC.npy")
validationVector = np.load("validateVectorGREC.npy")
with open('GraphHelperObjGREC', 'rb') as config_dictionary_file:
    graph = pickle.load(config_dictionary_file)

In [18]:
kmeans = KMeans(n_clusters=22, random_state=0).fit(trainVector)

In [24]:
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import homogeneity_score

pred = kmeans.predict(trainVector)
score = rand_score(y_train,pred)
print('Dunn Index Accuracy of Train:{0:f}'.format(score))
print("Homogenity Score of Train %.6f" % homogeneity_score(y_train, pred))

print()
print()
pred = kmeans.predict(validationVector)
score = rand_score(y_validate,pred)
print('Dunn Index Accuracy of Validation:{0:f}'.format(score))
print("Homogenity Score of Validation %.6f" % homogeneity_score(y_validate, pred))


print()
print()
pred = kmeans.predict(testVector)
score = rand_score(y_test,pred)
print('Dunn Index Accuracy of Test:{0:f}'.format(score))
print("Homogenity Score of Test %.6f" % homogeneity_score(y_test, pred))

Dunn Index Accuracy of Train:0.874985
Homogenity Score of Train 0.421175


Dunn Index Accuracy of Validation:0.866544
Homogenity Score of Validation 0.408794


Dunn Index Accuracy of Test:0.867661
Homogenity Score of Test 0.379925


In [20]:
from sklearn import svm
clf = svm.SVC()
clf.fit(trainVector, y_train)

SVC()

In [21]:
svmPredtrain = clf.predict(trainVector)
score = accuracy_score(y_train,svmPredtrain)
print('Accuracy:{0:f}'.format(score))

Accuracy:0.444056


In [22]:
svmPredvalidate = clf.predict(validationVector)
score = accuracy_score(y_validate,svmPredvalidate)
print('Accuracy Validate:{0:f}'.format(score))

svmPredtest = clf.predict(testVector)
score = accuracy_score(y_test,svmPredtest)
print('Accuracy Test:{0:f}'.format(score))

Accuracy Validate:0.318182
Accuracy Test:0.339015


In [23]:
print(testVector.shape)
print(validationVector.shape)
print(trainVector.shape)


(528, 10)
(286, 10)
(286, 10)
